<a href="https://colab.research.google.com/github/spi727/NeuralNetworks/blob/main/SMS_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.17.0-dev20240304


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-03-05 01:17:23--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2024-03-05 01:17:23 (9.56 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-03-05 01:17:23--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
names=['type','message']
train=pd.read_csv(train_file_path,names=names,sep="\t")
test=pd.read_csv(test_file_path,names=names,sep="\t")
train.dropna()
print(len(train))
print(len(test))

4179
1392


In [ ]:
replace={"ham":0,"spam":1}
train["type"]=train["type"].map(replace).fillna(train["type"])
test["type"]=test["type"].map(replace).fillna(test["type"])
train.head()

,type,message
0,0,ahhhh...just woken up!had a bad dream about u ...
1,0,you can never do nothing
2,0,"now u sound like manky scouse boy steve,like! ..."
3,0,mum say we wan to go then go... then she can s...
4,0,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
train_message=train["message"].values.tolist()
train_label=train["type"].values
test_message=test["message"].values.tolist()
test_label=test["type"].values
print(train_message)

['ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.', 'you can never do nothing', 'now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?', 'mum say we wan to go then go... then she can shun bian watch da glass exhibition...', 'never y lei... i v lazy... got wat? dat day ü send me da url cant work one...', 'in xam hall boy asked girl tell me the starting term for dis answer i can den manage on my own after lot of hesitation n lookin around silently she said the! intha ponnungale ipaditan;)', "genius what's up. how your brother. pls send his number to my skype.", 'they finally came to fix the ceiling.', 'urgent! call 09066350750 from your landline. your complimentary 4* ibiza holiday or 10,000 cash await collection sae t&cs po box 434 sk3 8wp 150 ppm 18+', 'now that you have started dont stop. just pray for more good ideas and anyt

In [ ]:
vocabulary={}
for message in train_message:
  for voc in message.split():
    if voc not in vocabulary:
      vocabulary[voc]=1
    else:
      vocabulary[voc]+=1

voclen=len(vocabulary)
max=200

11330


In [ ]:
encoded_train=[one_hot(m,voclen) for m in train_message]
padded_train=sequence.pad_sequences(encoded_train,maxlen=200)
encoded_test=[one_hot(m,voclen) for m in test_message]
padded_test=sequence.pad_sequences(encoded_test,maxlen=200)

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(voclen,32))
model.add(tf.keras.layers.LSTM(32))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history=model.fit(padded_train, train_label, validation_data=(padded_test, test_label), epochs=10)


Epoch 1/10
131/131 - 8s - 65ms/step - acc: 0.9201 - loss: 0.2584 - val_acc: 0.9763 - val_loss: 0.1143
Epoch 2/10
131/131 - 10s - 78ms/step - acc: 0.9864 - loss: 0.0657 - val_acc: 0.9799 - val_loss: 0.0624
Epoch 3/10
131/131 - 11s - 84ms/step - acc: 0.9928 - loss: 0.0321 - val_acc: 0.9878 - val_loss: 0.0528
Epoch 4/10
131/131 - 6s - 44ms/step - acc: 0.9959 - loss: 0.0172 - val_acc: 0.9835 - val_loss: 0.0582
Epoch 5/10
131/131 - 6s - 49ms/step - acc: 0.9981 - loss: 0.0100 - val_acc: 0.9871 - val_loss: 0.0546
Epoch 6/10
131/131 - 10s - 78ms/step - acc: 0.9995 - loss: 0.0050 - val_acc: 0.9849 - val_loss: 0.0572
Epoch 7/10
131/131 - 6s - 44ms/step - acc: 0.9990 - loss: 0.0050 - val_acc: 0.9885 - val_loss: 0.0543
Epoch 8/10
131/131 - 6s - 48ms/step - acc: 0.9995 - loss: 0.0032 - val_acc: 0.9878 - val_loss: 0.0622
Epoch 9/10
131/131 - 6s - 45ms/step - acc: 0.9998 - loss: 0.0019 - val_acc: 0.9878 - val_loss: 0.0647
Epoch 10/10
131/131 - 10s - 76ms/step - acc: 0.9998 - loss: 0.0015 - val_acc: 0

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  names={0:"ham",1:"spam"}
  e=[one_hot(pred_text,voclen)]
  p=sequence.pad_sequences(e,maxlen=200)
  prediction = [model.predict(p)[0][0], names[np.round(model.predict(p)[0][0])]]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[0.00022463773, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
You passed the challenge. Great job!
